In [40]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [41]:
#full pre-activation방식
def conv2_x(x):

    x1=layers.BatchNormalization()(x)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
  
    x2=layers.Add()([x,x1])
    x2=layers.ReLU()(x2)

    return x2

In [42]:
def conv3_x(x):
    x1=layers.Conv2D(filters=128,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=512,kernel_size=(1,1),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
  
    x2=layers.Add()([x,x1])
    x2=layers.ReLU()(x2)

    return x2

In [43]:
def conv4_x(x):
    x1=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=1024,kernel_size=(1,1),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
  
    x2=layers.Add()([x,x1])
    x2=layers.ReLU()(x2)

    return x2

In [44]:
def conv5_x(x):
    x1=layers.Conv2D(filters=512,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=2048,kernel_size=(1,1),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
  
    x2=layers.Add()([x,x1])
    x2=layers.ReLU()(x2)

    return x2

In [45]:
def ReductionBlock(x,f1,f2):
    
    x1=layers.Conv2D(filters=f1,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=f1,kernel_size=(3,3),strides=(2,2),padding='same')(x1)
    x1=layers.BatchNormalization()(x1) 
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=f2,kernel_size=(1,1),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)


    x2=layers.Conv2D(filters=f2,kernel_size=(1,1),strides=(2,2),padding='same')(x) #pretrained model 보고 convlayer 추정
    x2=layers.BatchNormalization()(x2)
   
    
    x3=layers.Add()([x1,x2])
    x3=layers.ReLU()(x3)
    return x3
    

In [46]:
input=Input(shape=(224,224,3))

x=layers.Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),padding='same')(input)
x=layers.BatchNormalization()(x)
x=layers.ReLU()(x)
x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)


x2=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same')(x)
x2=layers.BatchNormalization()(x2)

x1=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),padding='same')(x)
x1=layers.BatchNormalization()(x1)
x1=layers.ReLU()(x1)
x1=layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(x1)
x1=layers.BatchNormalization()(x) 
x1=layers.ReLU()(x)
x1=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same')(x1)
x1=layers.BatchNormalization()(x1)
x1=layers.ReLU()(x1)
x=layers.Add()([x1,x2])

x=conv2_x(x)
x=conv2_x(x)
print(x.shape)

x=ReductionBlock(x,128,512)
print(x.shape)
x=conv3_x(x)
x=conv3_x(x)
x=conv3_x(x)


x=ReductionBlock(x,256,1024)
x=conv4_x(x)
x=conv4_x(x)
x=conv4_x(x)
x=conv4_x(x)
x=conv4_x(x)

x=ReductionBlock(x,512,2048)
x=conv5_x(x)
x=conv5_x(x)

x=layers.GlobalAveragePooling2D()(x)
output=layers.Dense(units=1000,activation='softmax')(x)

model = tf.keras.models.Model(input,output)

(None, 56, 56, 256)
(None, 28, 28, 512)


In [ ]:
plot_model(model,to_file='ResNet_50.png',show_shapes=True)

In [47]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_525 (Conv2D)            (None, 112, 112, 64  9472        ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_546 (Batch  (None, 112, 112, 64  256        ['conv2d_525[0][0]']             
 Normalization)                 )                                                          

In [10]:
import tensorflow as tf
import numpy as np

(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

# Normalization
x_train=x_train/255.

x_test=x_test/255.

x_train.shape

x_train = tf.image.resize(x_train, [224,224]) 

x_test = tf.image.resize(x_test, [224,224]) 

y_train = tf.keras.utils.to_categorical(np.array(y_train))
y_test = tf.keras.utils.to_categorical(np.array(y_test))

In [11]:
model.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['acc'])

In [12]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=1,batch_size=8,verbose=1)

 809/6250 [==>...........................] - ETA: 3:02:41 - loss: 2.4175 - acc: 0.1562